### Extracts all data from each precision station for 2017-2019. Not all is used now, but allows for potential future analysis

In [ ]:
import requests
import pandas as pd
import io
import urllib.request
import gc
from time import strptime
import json
import calendar
from datetime import datetime

In [ ]:
## List of months
months = pd.date_range('2017-01-01','2020-03-13', 
              freq='MS').strftime("%Y%m").tolist()

In [ ]:
## Get Sensor Metadata, create sensor list
r = requests.get('https://newcastle.urbanobservatory.ac.uk/api/v1.1/sensors/csv/?broker=aq_mesh_api&search=mon')
sensorInfo = pd.read_csv(io.StringIO(r.text))
sensorList = sensorInfo["Sensor Name"].to_list()

In [ ]:
combBaseline = pd.DataFrame()

In [ ]:
## Call packaged into monthly chunks 
## Takes around 1hr20mins
print(datetime.now().strftime('%H:%M:%S'))
for s in sensorList:
    print(s)
    for m in months:      
        dataCall = 'https://newcastle.urbanobservatory.ac.uk/api/v1.1/sensors/'+s+'/data/csv/'
            
        startDate = m + '01'
        endDate = m + str(calendar.monthrange(int(m[:4]), int(m[-2:]))[1])

        params = dict(
                    starttime=startDate,
                    endtime=endDate
        )
            
        r = requests.get(dataCall,params)
            
        ## Check if API call succesfuly, merge into one dataframe 
        if r.status_code != 404:
            sensorData = pd.read_csv(io.StringIO(r.text))
            combBaseline = combBaseline.append(sensorData, ignore_index=True)
print(datetime.now().strftime('%H:%M:%S'))

In [ ]:
## Pickle out
pd.to_pickle(combBaseline, '../cache/baseline-airquality-airmon.pkl')
gc.collect()